In [1]:
import pandas
from IPython.display import clear_output

In [2]:
from classes import Question
import classes
import look_up_utils
import createq_utils
from display_utils import display_question

## Set parameter values

In [3]:
from glob import glob
import os
frame_paths = [os.path.basename(path)
              for path in glob('../EventRegistries/GunViolenceArchive/frames/mass*')
              if not 'Icon' in path
             ]

add_fr = False
if add_fr:
    frame_paths.append('../EventRegistries/FireRescue1/firerescue.pickle')

print(frame_paths)

df = pandas.concat([pandas.read_pickle('../EventRegistries/GunViolenceArchive/frames/' + frame)
                    for frame in frame_paths])
df = df.drop_duplicates(subset='incident_uri')

confusion_tuple = ('location', 'participant')
min_num_answer_incidents = 2

['mass_shootings', 'mass_shootings_2013', 'mass_shootings_2014', 'mass_shootings_2015']


## Load frames

## Load look_up

In [4]:
look_up, parameters2incident_uris = look_up_utils.create_look_up(df, 
                                                                 discard_ambiguous_names=True,
                                                                 allowed_incident_years={2013, 2014, 2015, 2016, 2017},
                                                                 check_name_in_article=True)

## Load questions

In [5]:
candidates=createq_utils.lookup_and_merge(look_up, 
                                          parameters2incident_uris,
                                          confusion_tuple,
                                          min_num_answer_incidents,
                                          df,
                                          debug=False,
                                          inspect_one=False,
                                          set_attr_values=True) 
print({c.granularity for c in candidates})

{('state', 'last'), ('state', 'first'), ('city', 'first'), ('city', 'last')}


## Display a question

In [6]:
print('the available attributes are:\n')
for candidate in candidates:
    for prop in dir(candidate):
        if all([not prop.startswith('_'),
                prop not in {'debug', 'confusion_df', 'answer_df'}]):
            print(prop)
    break

the available attributes are:

a_avg_date_spread
a_avg_num_sources
a_sources
answer
answer_incident_uris
c2s_ratio
c_avg_date_spread
c_avg_num_sources
c_sources
confusion_factors
confusion_incident_uris
gold_loc_meaning
granularity
gvdb_annotations
loc_oa
location_confusion
meanings
num_a_sources
num_both_sf_overlap
num_c_sources
num_gvdb_part_annotations
oa
oa_info
participant_confusion
q_id
question
set_all_attributes
sf
time_confusion
time_oa
to_conll


In [ ]:
wanted_attrs = [
    'q_id',
    'question',
    'granularity',
    'answer_incident_uris',
    'num_both_sf_overlap',
    #'confusion_incident_uris',
    'meanings',
    'answer',
    'c2s_ratio',
    'location_confusion',
    'time_confusion',
    'participant_confusion',
    'a_avg_num_sources',
    'a_avg_date_spread',
    'c_avg_num_sources',
    'c_avg_date_spread',
    'num_gvdb_part_annotations',
    'loc_oa',
    'time_oa'
]

for candidate in candidates:
    clear_output()
    display_question(candidate, wanted_attrs)
    break
    #input('next?')

,0
q_id,60
question,"How many killing AND injuring events happened in ('Illinois',) (state) that involve the name Brown (last) ?"
granularity,"('state', 'last')"
answer_incident_uris,"{'676686', '422064'}"
num_both_sf_overlap,2
meanings,"defaultdict(, {(('Illinois',), ('676686', 'Brown')): {'676686'}, (('Illinois',), ('422064', 'Brown')): {'422064'}})"
answer,2
c2s_ratio,68.0
location_confusion,True
time_confusion,False


In [ ]:
for candidate in candidates:
    candidate.to_conll(candidate.answer_df)
    candidate.to_conll(candidate.confusion_df)